In [ ]:
#Cell 1
"""
UNIVERSITY OF SYDNEY

PHYS2921 SSP Research Project: Time-periodic Quantum State Transfer With Topological Edge States

Code Author: NYAN LIN HTUN

Research Supervisor: RADITYA BOMANTARA

2021 Semester 1
"""

In [1]:
#Cell 2
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from IPython.display import clear_output
%matplotlib qt

In [2]:
#Cell 3

#Defining SSH Hamiltonian
def ssh_hamiltonian(v,w,N):
    """
    v is m,a to m,b
    w is m,b to m+1,a
    N is number of cells
    """
    dim = N*2
    hmatrix = np.zeros((dim,dim))+0j
    """
    if v != 0:
        for m in range(1,N+1):
            am = np.zeros(dim)+0j
            bm = np.zeros(dim)+0j
            am[int(m)*2-2] = 1
            bm[int(m)*2-1] = 1
            hmatrix += v*(np.outer(bm,am)+np.outer(am,bm))
    if w != 0:
        for m in range(1,N):
            amp1 = np.zeros(dim)+0j
            bm = np.zeros(dim)+0j
            amp1[(int(m)+1)*2 -2] = 1
            bm[int(m)*2-1] = 1
            hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm))
    """
    #Maybe this is faster
    for i in range(1,dim):
        if i % 2 != 0:
            hmatrix[i-1][i] = v
            hmatrix[i][i-1] = v
        else:
            hmatrix[i-1][i] = w
            hmatrix[i][i-1] = w
    
    return hmatrix
    
def time_evolution(v,w,N,T):
    return np.matmul(expm(-1j*ssh_hamiltonian(0,w,N)*T/2),expm(-1j*ssh_hamiltonian(v,0,N)*T/2))

#Adiabatic Part II Hamiltonian
def hamiltonian2i(w,N,i): #####FIX: j jumps by two
    dim = N*2
    hmatrix = np.zeros((dim,dim))+0j
    #m is the j used in equations
    first_m_range = np.linspace(1,i-1,int(i*0.5)) #WRONG: np.linspace(1,i-1,int((i-1)*0.5))
    second_m_range = np.linspace(i+1,N-1,N-i-1)
    for m in first_m_range:
        am = np.zeros(dim)+0j
        bm = np.zeros(dim)+0j
        amp1 = np.zeros(dim)+0j
        bmp1 = np.zeros(dim)+0j
        am[int(m)*2-2] = 1
        bm[int(m)*2-1] = 1
        amp1[(int(m)+1)*2 -2] = 1
        bmp1[(int(m)+1)*2 -1] = 1
        hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm)+np.outer(am,bmp1)+np.outer(bmp1,am))
    
    for m in second_m_range:
        bm = np.zeros(dim)+0j
        amp1 = np.zeros(dim)+0j
        bm[int(m)*2-1] = 1
        amp1[(int(m)+1)*2 -2] = 1
        hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm))
    
    return hmatrix

def interpolated_time_evolution(s,v,w,N,i,T):
    H1 = ssh_hamiltonian(v,0,N)
    H2 = ssh_hamiltonian(0,w,N)
    H2i = hamiltonian2i(w,N,i)
    return np.matmul(expm(-1j*(s*H2i + (1-s)*H2)*T/2),expm(-1j*H1*T/2))

#Interpolated Hamiltonian for Direct Protocol: hamiltonian2i(w,N,N/2)

In [ ]:
#Cell 4

#Existence and robustness of topological states, fixed v varying w. Graph of quasienergy
T = 2
v = 1
N = 50
w_range = np.linspace(0,2*np.pi,100)
plt.figure()
for w in w_range:
    U_t = time_evolution(v,w,N,T)
    #eps_array = np.log(np.linalg.eigvalsh(U_t))
    eps_array = np.angle(np.linalg.eigvals(U_t))*-1
    plt.plot(w*np.ones(len(eps_array)),eps_array,'o',color='black') #Use plot instead of scatter
    clear_output(wait=True)
    print("Plotting quasienergies: "+str(w*100/(4*np.pi))+"% complete!")
for w in w_range:
    U_t = time_evolution(v,w,N,T)
    eigboth = np.linalg.eig(U_t)
    eigval_arr = eigboth[0]
    eigvec_arr = eigboth[1]
    left_index = np.where(np.abs(eigvec_arr[0])**2+np.abs(eigvec_arr[1])**2+np.abs(eigvec_arr[2])**2+np.abs(eigvec_arr[3])**2 >0.2)
    right_index = np.where(np.abs(eigvec_arr[-1])**2+np.abs(eigvec_arr[-2])**2+np.abs(eigvec_arr[-3])**2+np.abs(eigvec_arr[-4])**2 >0.2)
    left_eigval_exp = eigval_arr[left_index[0]]
    left_eigval = np.angle(left_eigval_exp)*-1
    right_eigval_exp = eigval_arr[right_index[0]]
    right_eigval = np.angle(right_eigval_exp)*-1
    
    if left_eigval != [] :
        plt.plot(w,left_eigval[0],'bo') #'ro'
    if right_eigval != []:
        plt.plot(w,right_eigval[0],'bo')
    
    clear_output(wait=True)
    print("Plotting quasienergies: 100% complete!")
    print("Plotting edge states quasienergies: "+str(w*100/(2*np.pi))+"% complete!")
plt.xlabel('w')
plt.ylabel('\u03B5')
plt.title('v = 1')
plt.show()


In [3]:
#Cell 5

#Existence and robustness of topological states, fixed w varying v. Graph of quasienergy
T = 2
N = 50
w = 1
v_range = np.linspace(0,4*np.pi,100)
plt.figure()
for v in v_range:
    U_t = time_evolution(v,w,N,T)
    #eps_array = np.log(np.linalg.eigvalsh(U_t))
    eps_array = np.angle(np.linalg.eigvals(U_t))*-1
    plt.plot(v*np.ones(len(eps_array)),eps_array,'o',color='black')
    clear_output(wait=True)
    print("Plotting quasienergies: "+str(v*100/(4*np.pi))+"% complete!")
for v in v_range:
    U_t = time_evolution(v,w,N,T)
    eigboth = np.linalg.eig(U_t)
    eigval_arr = eigboth[0]
    eigvec_arr = eigboth[1]
    left_index = np.where(np.abs(eigvec_arr[0])**2+np.abs(eigvec_arr[1])**2+np.abs(eigvec_arr[2])**2+np.abs(eigvec_arr[3])**2 >0.4)
    right_index = np.where(np.abs(eigvec_arr[-1])**2+np.abs(eigvec_arr[-2])**2+np.abs(eigvec_arr[-3])**2+np.abs(eigvec_arr[-4])**2 >0.4)
    left_eigval_exp = eigval_arr[left_index[0]]
    left_eigval = np.angle(left_eigval_exp)*-1
    right_eigval_exp = eigval_arr[right_index[0]]
    right_eigval = np.angle(right_eigval_exp)*-1
    
    if left_eigval != [] :
        plt.plot(v,left_eigval[0],'ro')
    if right_eigval != []:
        plt.plot(v,right_eigval[0],'bo')
    
    clear_output(wait=True)
    print("Plotting quasienergies: 100% complete!")
    print("Plotting edge states quasienergies: "+str(v*100/(4*np.pi))+"% complete!")
plt.xlabel('v')
plt.ylabel('\u03B5')
plt.title('w = 1')
plt.show()


Plotting quasienergies: 100% complete!
Plotting edge states quasienergies: 100.0% complete!


In [11]:
#Cell 6

#2. Adiabatic transfer of edge states (Direct Protocol). Graph of quasienergy against interpolation:
v = 
w = 
N = 
T = 
i = int(N/2)
s_space = np.linspace(0,1,100) #Change the third argument (number of intervals)

#Plotting
plt.figure()
for s in s_space:
    #Finding and plotting eigenvalues of U'
    U_s = interpolated_time_evolution(s,v,w,N,i,T)
    eps_array = np.angle(np.linalg.eigvals(U_s))*-1
    plt.plot(s*np.ones(len(eps_array)),eps_array,'o',color='black')
    
    #Finding and plotting eigenvalues of edge states:
    eigboth = np.linalg.eig(U_s)
    eigval_arr = eigboth[0]
    eigvec_arr = eigboth[1]
    if s < 0.5:
        left_index = np.where(np.abs(eigvec_arr[0])**2+np.abs(eigvec_arr[1])**2 >0.9)
    else:
        left_index = np.where(np.abs(eigvec_arr[(i+2)*2 -4])**2+np.abs(eigvec_arr[(i+2)*2 -3])**2 >0.9)
    right_index = np.where(np.abs(eigvec_arr[-1])**2+np.abs(eigvec_arr[-2])**2 >0.9)
    left_eigval_exp = eigval_arr[left_index[0]]
    left_eigval = np.angle(left_eigval_exp)*-1
    right_eigval_exp = eigval_arr[right_index[0]]
    right_eigval = np.angle(right_eigval_exp)*-1
    if len(left_eigval) != 0 :
        plt.plot(s,left_eigval[0],'ro')
    if len(right_eigval) != 0:
        plt.plot(s,right_eigval[0],'bo')
    clear_output(wait=True)
    print(str(s*100)+"% complete!")
plt.xlabel('s')
plt.ylabel('\u03B5')
plt.title('N = '+str(N))
plt.show()

100.0% complete!


In [ ]:
#Cell 7

#Adiabatic transfer of edge states (Direct Protocol). Graph of fidelity (with |A,N/2+1>) against interpolation:
#Prints final fidelity in output
v = 
w = 
N = 
T = 
i = int(N/2)
s_space = np.linspace(0,1,100) #Represents interpolation completion time s. Change the third argument (number of intervals) to adjust \Delta s

a1 = np.zeros(2*N)+0j
a1[0] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_s = a1

finalfid = 0
sarr = np.zeros(int(len(s_space)/2))
fidarr = np.zeros(int(len(s_space)/2))

for s in s_space:
    psi_s = np.matmul(interpolated_time_evolution(s,v,w,N,i,T),psi_s)
    fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
    if int(np.where(s_space == s)[0]) % 2 != 0:
        sarr[int((np.where(s_space == s)[0]+1)/2) -1] = s
        fidarr[int((np.where(s_space == s)[0]+1)/2) -1] = fid
    finalfid = fid

plt.figure()#    
plt.plot(sarr,fidarr)
plt.xlabel('s')
plt.ylabel(r'$|\langle\psi(s)| \Psi\rangle|$')
plt.ylim(-0.05,1.05)
plt.title('N = '+str(N))
plt.show()#
print(finalfid)

In [ ]:
#Cell 8

#Adiabatic transfer of edge states (Direct Protocol). Graph of fidelity (with (|A,N/2+1> + |B,N/2+1>)/sqrt(2)) against interpolation:
#Prints final fidelity in output
v = 
w = 
N = 
T = 
i = int(N/2)
s_space = np.linspace(0,1,100) #Represents interpolation completion time s. Change the third argument (number of intervals) to adjust \Delta s

start_state = np.zeros(2*N)+0j
start_state[0] = 1
start_state[1] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_const[2*(int(N/2) +1)-1] = 1 #state |A,N/2+1> + |B,N/2+1>
psi_const *= 1/np.sqrt(2)
psi_s = start_state * 1/np.sqrt(2)

finalfid = 0
sarr = np.zeros(int(len(s_space)/2))
fidarr = np.zeros(int(len(s_space)/2))

for s in s_space:
    psi_s = np.matmul(interpolated_time_evolution(s,v,w,N,i,T),psi_s)
    fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
    #plt.plot(s,fid,'bo')
    if int(np.where(s_space == s)[0]) % 2 != 0:
        sarr[int((np.where(s_space == s)[0]+1)/2) -1] = s
        fidarr[int((np.where(s_space == s)[0]+1)/2) -1] = fid
    finalfid = fid

plt.figure()
plt.plot(sarr,fidarr)
plt.xlabel('s')
plt.ylabel(r'$|\langle\psi(s)| \Psi\rangle|$') #Run cell 2 first. Afterwards, set usetex to false
plt.ylim(-0.05,1.05)
plt.title('N = '+str(N))
plt.show()#
print(finalfid)

In [19]:
#Cell 9

#3. Adiabatic transfer of edge states (Step-by-step Protocol). Graph of quasienergies against interpolations:
v = 
w = 
T = 
N = 
s_space = np.linspace(0,1,100) #Represents interpolation completion time s. Change the third argument (number of intervals) to adjust \Delta s
add_i = # rate of interpolation x 2 ;add_i=2 for 1 interpolation, add_i=4 for 2, ...
i_space = range(add_i,int(N/2) +1,add_i) #Starts at i=2, increments by 2, ends at i = N/2
H1 = ssh_hamiltonian(v,0,N)
H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T
plt.figure()
for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        #Constructing U and finding eigenvalues
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        eps_array = np.angle(np.linalg.eigvals(U_s))*-1
        plt.plot((i-add_i+add_i*s)*np.ones(len(eps_array)),eps_array,'o',color='black')

        #Finding and plotting eigenvalues of edge states:
        eigboth = np.linalg.eig(U_s)
        eigval_arr = eigboth[0]
        eigvec_arr = eigboth[1]
        if s < 0.5:
            left_index = np.where(np.abs(eigvec_arr[i*2 -4])**2+np.abs(eigvec_arr[i*2 -3])**2 >0.95)
        else:
            left_index = np.where(np.abs(eigvec_arr[(i+2)*2 -4])**2+np.abs(eigvec_arr[(i+2)*2 -3])**2 >0.95)
        right_index = np.where(np.abs(eigvec_arr[-1])**2+np.abs(eigvec_arr[-2])**2 >0.95)
        left_eigval_exp = eigval_arr[left_index[0]]
        left_eigval = np.angle(left_eigval_exp)*-1
        right_eigval_exp = eigval_arr[right_index[0]]
        right_eigval = np.angle(right_eigval_exp)*-1
        if len(left_eigval) != 0 :
            plt.plot((i-add_i+add_i*s),left_eigval[0],'ro') 
        if len(right_eigval) != 0:
            plt.plot((i-add_i+add_i*s),right_eigval[0],'bo') 
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")
plt.xlabel('i')
plt.ylabel(r'$\epsilon$')
plt.title('N = '+str(N)+'')
plt.show()

100.0% complete!


In [ ]:
#Cell 10

#Adiabatic transfer of edge states (Step-by-step). Graph of fidelity (with |A,N/2+1>) against interpolation:
#Prints final fidelity in output
v = 
w = 
T = 
N = 
s_space = np.linspace(0,1,100) #Range of interpolation completion time s
add_i = # rate of interpolation x 2 
i_space = range(add_i,int(N/2) +1,add_i) 
H1 = ssh_hamiltonian(v,0,N)
H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T

a1 = np.zeros(2*N)+0j
a1[0] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_s = a1

finalfid = 0
iarr = np.zeros(int(len(s_space)/2 * len(i_space)))
fidarr = np.zeros(int(len(s_space)/2 * len(i_space)))
#iarr = []
#fidarr = []
fidcheckarr = []
arr_index = 0

for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        psi_s = np.matmul(U_s,psi_s)
        fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
        if int(np.where(s_space == s)[0]) % 2 != 0:
            iarr[arr_index] = i-add_i+add_i*s 
            fidarr[arr_index] = fid 
            arr_index +=1
        finalfid = fid
            
        #fidcheckarr.append(fid)
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")

plt.figure()#
plt.plot(iarr,fidarr)
plt.ylim(-0.05,1.05)
plt.xlabel('i')
plt.ylabel('|<\u03C8(s)|A,N/2+1>|')
plt.title('N = '+str(N))
plt.show()
print(finalfid)

In [ ]:
#Cell 11

#Adiabatic transfer of edge states (Step-by-step). Graph of fidelity (with (|A,N/2+1> + |B,N/2+1>)/sqrt(2)) against interpolation:
#Prints final fidelity in output
v = 
w = 
T = 
N = 
s_space = np.linspace(0,1,100) #Range of interpolation completion time s
add_i = # rate of interpolation x 2
i_space = range(add_i,int(N/2) +1,add_i) 
H1 = ssh_hamiltonian(v,0,N)
H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T

start_state = np.zeros(2*N)+0j
start_state[0] = 1
start_state[1] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_const[2*(int(N/2) +1)-1] = 1 #state |A,N/2+1> + |B,N/2+1>
psi_const *= 1/np.sqrt(2)
psi_s = start_state * 1/np.sqrt(2)

finalfid = 0
iarr = np.zeros(int(len(s_space)/2 * len(i_space)))
fidarr = np.zeros(int(len(s_space)/2 * len(i_space)))
#iarr = []
#fidarr = []
fidcheckarr = []
arr_index = 0

for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        psi_s = np.matmul(U_s,psi_s)
        fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
        if int(np.where(s_space == s)[0]) % 2 != 0:
            iarr[arr_index] = i-add_i+add_i*s 
            fidarr[arr_index] = fid 
            arr_index +=1
        finalfid = fid
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")

plt.figure()
plt.plot(iarr,fidarr)
plt.xlabel('i')
plt.ylabel('| <\u03C8(s) | \u03A8> |')
plt.title('N = '+str(N))
plt.show()
print(finalfid)

In [ ]:
#Cell 12

#Finding the optimal interpolation rate. Only prints the final fidelity (with |A,N/2+1>) in output:
#Record the data in an array in the next cell with another array containing corresponding interpolation rate.
v = 
w = 
T = 
N = 
add_i = #rate of interpolation x 2
totalT = #Total time of whole protocol. Usually, I use N/4 x 100; N/4 is number of interpolations for step-by-step, 100 is for \Delta s = 0.01
i_space = range(add_i,int(N/2) +1,add_i)
s_space = np.linspace(0,1,int(totalT/len(i_space)))
H1 = ssh_hamiltonian(v,0,N)
H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T

a1 = np.zeros(2*N)+0j
a1[0] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_s = a1

finalfid = 0
iarr = np.zeros(int(len(s_space)/2 * len(i_space)))
fidarr = np.zeros(int(len(s_space)/2 * len(i_space)))
arr_index = 0

for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        psi_s = np.matmul(U_s,psi_s)
        fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
        if int(np.where(s_space == s)[0]) % 2 != 0:
            iarr[arr_index] = i-add_i+add_i*s 
            fidarr[arr_index] = fid 
            arr_index +=1
        finalfid = fid
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")

plt.figure()#
plt.plot(iarr,fidarr)
plt.ylim(-0.05,1.05)
plt.xlabel('i')
plt.ylabel('|<\u03C8(s)|A,N/2+1>|')
plt.title('N = '+str(N)+', interpolation rate = '+str(int(add_i/2)))
plt.show()
print(finalfid)

In [21]:
#Cell 13

#Plotting final fidelity for optimization. Uncomment one of the data sets to plot final fidelity w.r.t. interpolation rate
#Prints interpolation rate with highest final fidelity.
#NOTE: rate_arr is array of interpolation rate, so each entry is add_i/2
'''
N=40
totalTime=2000
add_i=2 :
add_i=4 :
add_i=10:0.9999989966650702
add_i=20:0.9999998748545902 (Largest,N/2)

totalTime=3000
add_i=10:0.9999995551423434
add_i=20:0.9999999444022175 (Largest, N/2)

totalTime=4000
add_i=2 :
add_i=4 :
add_i=10:0.9999997497899091
add_i=20:0.9999999687349761 (Largest,N/2)

totalTime=6000
add_i=10:0.9999998887865887
add_i=20:0.9999999861035145 (Largest,N/2)
'''

'''
N=80
totalTime=2000 *
add_i=8 :0.9999843655625229
add_i=10:0.9999919780614038
add_i=20:0.9999989985922991 (Largest,N/4)
add_i=40:0.9999978207498246

totalTime=4000
add_i=8 :
add_i=10:0.9999979933311575
add_i=20:0.9999997497091977
add_i=40:0.9999999687391378 (Largest,N/2)

totalTime=6000
add_i=20:
add_i=40:
'''
'''
N=160
totalTime=2000
add_i=10:0.9999360232992154
add_i=16:0.9999834319553241 (Largest,N/10)
add_i=20:0.999967445177948 
add_i=40:0.9959813743711203 (N/4)
add_i=80:

totalTime = 4000 *
add_i=16:0.9999960796656565
add_i=20:0.9999979971855956 (Largest,N/8)
add_i=40:0.9999956415043997
rate_arr = np.array([8,10,20])
finalfidarr = np.array([0.9999960796656565,0.9999979971855956,0.9999956415043997])
'''

'''
N=240
totalTime=2000
add_i=2  : #Not allowed
add_i=4  : #Not allowed
add_i=6  :
add_i=8  : #Not allowed
add_i=10 : #Not allowed
add_i=12 :
add_i=20 : #Not allowed
add_i=24 :
add_i=30 :
add_i=40 : #Not allowed
add_i=60 :
add_i=120:

totalTime=4000
add_i=2  : #Not allowed
add_i=4  : #Not allowed
add_i=6  :
add_i=8  : #Not allowed
add_i=10 : #Not allowed
add_i=12 :
add_i=20 : #Not allowed
add_i=24 :
add_i=30 :
add_i=40 : #Not allowed
add_i=60 :
add_i=120:

totalTime=6000 *
rate_arr = np.array([1,2,3,4,5,6,10,12,15,20,30,60])
finalfidarr = np.array([0.9969754560094898,0.9996245853292367,0.9998883121405672,0.9999530974212822,0.9999759343776828,0.9999860713292483,0.9999969957802495,0.9999982556775926,0.999998942216455,0.9999934622640932,0.9997012538522518,0.9852263013031914])
#N/8
'''

'''
N=480
totalTime=6000
finalfidarr = np.array([0.9998903005212003,0.9999502966898896,0.9999023387136283,0.9995866881688835,0.9978411625455125,0.9879925062713356,0.9713383392925352,0.9325604383195175])
rate_arr = np.array([6,8,10,12,15,20,24,30])
'''

plt.figure()
plt.plot(rate_arr,finalfidarr)
maxfid = np.max(finalfidarr)
maxind = np.where(finalfidarr==maxfid)
plt.plot(rate_arr[maxind],maxfid,'ro') #Red dot shows largest final fidelity
print('Interpolation rate with highest fidelity =', rate_arr[maxind])
plt.plot(rate_arr[np.where(rate_arr==int(N/16))],finalfidarr[np.where(rate_arr==int(N/16))],'bo') #blue dot shows where interpolation rate = N/16
print('N/16 =', int(N/16))
plt.xlabel('interpolation rate')
plt.ylabel('Final fidelity')
plt.title('N = '+str(N))
plt.show()

Interpolation rate with highest fidelity = [10]
N/16 = 10


In [ ]:
#Find for the optimal n and vary w (fixed throughout). If sufficient time, do for other N
#4. Wrap Up, 2:
v = np.pi/2
w_dev = 0.1
w = np.pi +w_dev
T = 2
N = 48 #Must be a multiple of 
s_space = np.linspace(0,1,100)
n_optimal = int(N/8)#optimal n from previous cell
i_space = range(n_optimal,int(N/2) +1,n_optimal) #Starts at i=2, increments by 2, ends at i = N/2
H1 = ssh_hamiltonian(v,0,N)
H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T
plt.figure()
for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        #Constructing U and finding eigenvalues
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        eps_array = np.angle(np.linalg.eigvals(U_s))*-1
        plt.plot((i-2+n_optimal*s)*np.ones(len(eps_array)),eps_array,'o',color='black')
 
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")
plt.xlabel('i')
plt.ylabel('\u03B5')
plt.title('N = '+str(N) +' \u0394w = '+str(w_dev))
plt.show()

In [ ]:
#Cell 14

#Parameter. Varies v and w randomly. Plots quasienergy against complete interpolations
p_wdev_range = #Positive deviation range for w. Eg. p_wdev_range=0.1, w=pi -> pi-0.1<= w<= pi+0.1
v = np.pi/2
w = np.pi
T = 2
N = 240
n_optimal = int(N/8)#optimal n from previous cell
i_space = range(n_optimal,int(N/2) +1,n_optimal) #Starts at i=2, increments by 2, ends at i = N/2
s_space = np.linspace(0,1,int(totalT/len(i_space)))


a1 = np.zeros(2*N)+0j
a1[0] = 1
psi_const = np.zeros(2*N)+0j
psi_const[2*(int(N/2) +1)-2] = 1 #state |A,N/2+1>
psi_s = a1

finalfid = 0
iarr = np.zeros(int(len(s_space)/2 * len(i_space)))
fidarr = np.zeros(int(len(s_space)/2 * len(i_space)))
arr_index = 0

for i in i_space:
    for s in s_space:
        H1 = ssh_hamiltonian(v,0,N)
        H_start = ssh_hamiltonian(0,w,N) #H_0 for T/2<=t<T
        H_end = hamiltonian2i(w,N,i)
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        psi_s = np.matmul(U_s,psi_s)
        fid = np.abs(np.matmul(psi_s.transpose(),psi_const))
        if int(np.where(s_space == s)[0]) % 2 != 0:
            iarr[arr_index] = i-n_optimal+n_optimal*s 
            fidarr[arr_index] = fid 
            arr_index +=1
        finalfid = fid
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")

plt.figure()#
plt.plot(iarr,fidarr)
plt.ylim(-0.05,1.05)
plt.xlabel('i')
plt.ylabel('|<\u03C8(s)|A,N/2+1>|')
plt.title('N = '+str(N)+', interpolation rate = '+str(int(n_optimal/2)))
plt.show()
print(finalfid)

"""plt.figure()
for i in i_space:
    H_end = hamiltonian2i(w,N,i)
    for s in s_space:
        #Constructing U and finding eigenvalues
        U_s = np.matmul(expm(-1j*(s*H_end + (1-s)*H_start)*T/2),expm(-1j*H1*T/2))
        eps_array = np.angle(np.linalg.eigvals(U_s))*-1
        plt.plot((i-2+n_optimal*s)*np.ones(len(eps_array)),eps_array,'o',color='black')
 
    H_start = H_end
    clear_output(wait=True)
    print(str(i/(N/2) *100)+"% complete!")
plt.xlabel('i')
plt.ylabel('\u03B5')
plt.title('N = '+str(N) +' \u0394w = '+str(w_dev))
plt.show()"""

In [14]:
#Cell 

#Spatial Disorder. Hamiltonians has tiny deviations. Cell is for redefining functions. This cell can be run without Cell 3
#Defining SSH Hamiltonian
def ssh_hamiltonian(v,w,N,dev):
    """
    v is m,a to m,b
    w is m,b to m+1,a
    N is number of cells
    """
    dim = N*2
    hmatrix = np.zeros((dim,dim))+0j
    """
    if v != 0:
        for m in range(1,N+1):
            am = np.zeros(dim)+0j
            bm = np.zeros(dim)+0j
            am[int(m)*2-2] = 1
            bm[int(m)*2-1] = 1
            hmatrix += v*(np.outer(bm,am)+np.outer(am,bm))
    if w != 0:
        for m in range(1,N):
            amp1 = np.zeros(dim)+0j
            bm = np.zeros(dim)+0j
            amp1[(int(m)+1)*2 -2] = 1
            bm[int(m)*2-1] = 1
            hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm))
    """
    #Maybe this is faster
    for i in range(1,dim):
        if i % 2 != 0:
            hmatrix[i-1][i] = v + (np.random.rand()*2-1)*dev
            hmatrix[i][i-1] = v + (np.random.rand()*2-1)*dev
        else:
            hmatrix[i-1][i] = w + (np.random.rand()*2-1)*dev
            hmatrix[i][i-1] = w + (np.random.rand()*2-1)*dev
    
    return hmatrix
    
def time_evolution(v,w,N,T,dev):
    return np.matmul(expm(-1j*ssh_hamiltonian(0,w,N,dev)*T/2),expm(-1j*ssh_hamiltonian(v,0,N,dev)*T/2))

#Adiabatic Part II Hamiltonian
def hamiltonian2i(w,N,i): #####FIX: j jumps by two
    dim = N*2
    hmatrix = np.zeros((dim,dim))+0j
    #m is the j used in equations
    first_m_range = np.linspace(1,i-1,int(i*0.5)) #WRONG: np.linspace(1,i-1,int((i-1)*0.5))
    second_m_range = np.linspace(i+1,N-1,N-i-1)
    for m in first_m_range:
        am = np.zeros(dim)+0j
        bm = np.zeros(dim)+0j
        amp1 = np.zeros(dim)+0j
        bmp1 = np.zeros(dim)+0j
        am[int(m)*2-2] = 1 + (np.rand()*2-1)*dev/w
        bm[int(m)*2-1] = 1 + (np.rand()*2-1)*dev/w
        amp1[(int(m)+1)*2 -2] = 1 + (np.rand()*2-1)*dev/w
        bmp1[(int(m)+1)*2 -1] = 1 + (np.rand()*2-1)*dev/w
        hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm)+np.outer(am,bmp1)+np.outer(bmp1,am))
    
    for m in second_m_range:
        bm = np.zeros(dim)+0j
        amp1 = np.zeros(dim)+0j
        bm[int(m)*2-1] = 1 + (np.rand()*2-1)*dev/w
        amp1[(int(m)+1)*2 -2] = 1 + (np.rand()*2-1)*dev/w
        hmatrix += w*(np.outer(bm,amp1)+np.outer(amp1,bm))
    
    return hmatrix

def interpolated_time_evolution(s,v,w,N,i,T):
    H1 = ssh_hamiltonian(v,0,N)
    H2 = ssh_hamiltonian(0,w,N)
    H2i = hamiltonian2i(w,N,i)
    return np.matmul(expm(-1j*(s*H2i + (1-s)*H2)*T/2),expm(-1j*H1*T/2))

In [ ]:
#Cell 

#Spatial Disorder. Hamiltonians has tiny deviations. Prerequisite: Cell 

In [20]:
print(ssh_hamiltonian(3.14,0,4,0.1))

[[ 0.        +0.j  3.09256766+0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 3.08336531+0.j  0.        +0.j -0.04957792+0.j  0.        +0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j -0.03410324+0.j  0.        +0.j  3.17470712+0.j
   0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  3.05357903+0.j  0.        +0.j
  -0.01127321+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.05551393+0.j
   0.        +0.j  3.16157815+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   3.09477599+0.j  0.        +0.j  0.093592  +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   0.        +0.j  0.01226116+0.j  0.        +0.j  3.21405246+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
   